# Source/Target Specific Global Regressors

In [1]:
from py2neo import Node, Relationship, Graph
from numpy.random import rand
import numpy as num
from bokeh.plotting import output_notebook,figure, show
from bokeh.layouts import row,column,gridplot
from bokeh.models import Label
import numpy as np
from curve import *
from bokeh.models import Span,Legend
output_notebook()

Loading BokehJS ...

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import pickle as pk
A = pk.load(open('global.pk','rb'))

# sample nodes
N = ['L4','L34','L79','L123']

import random

def gen_hex_colour_code():
   return '#'+''.join([random.choice('0123456789ABCDEF') for x in range(6)])

C = {}

In [4]:
l = len(A)
D = {}
for i in range(l):
    D[i] = {}
    D[i]['n1'] = A[i]['n1']
    D[i]['n2'] = A[i]['n2']
    tmp = A[i]['envelope']
    ce = num.sum([xx[1] for xx in A[i]['envelope']])/len(A[i]['envelope'])
    cex = num.sum([xx[0] for xx in A[i]['envelope']])/len(A[i]['envelope'])
    D[i]['x'] = [xx[0]-cex for xx in tmp]
    D[i]['y'] = [xx[1]-ce for xx in tmp]
    D[i]['z'] = [xx-ce for xx in A[i]['z']]
    D[i]['slope'] = A[i]['slope']
    D[i]['r2'] = A[i]['r2']
p = []
C = {}
N = ['L'+str(i+1) for i in range(180)]
R = {}
for roi in N:
    # source taget
    datax = [xx['x'] for xx in D.values() if xx['n1'] == roi]
    datay = [xx['y'] for xx in D.values() if xx['n1'] == roi]
    fx = [item for sublist in datax for item in sublist]
    fy = [item for sublist in datay for item in sublist]
    F = lslinear(fx,fy)
    R["s-"+roi]=(F['r2'],F['slope'])
    
    
    # target
    datax = [xx['x'] for xx in D.values() if xx['n2'] == roi]
    datay = [xx['y'] for xx in D.values() if xx['n2'] == roi]
    fx = [item for sublist in datax for item in sublist]
    fy = [item for sublist in datay for item in sublist]
    F = lslinear(fx,fy)
    R["t-"+roi]=(F['r2'],F['slope'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:785: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


# Step 1: Adding noise
We consider X->Y pair with R2>90% to be ground truth data for slope estimation. Then the envelope points are contaminated by adding white noise to both their x and y coordinated, i.e., connection strength is added to white noise **normal(m=-2,var=1)** and distance is added to white noise **normal(m=0,var=10)**. After contaminating the data, ofcourse slope of envelope points will deviate from the groundtruth slope. Below the histogram for these deviations are plotted. Also R2 will significatly drop.

In [30]:
i = 0
ff = lambda x: x+ np.random.normal(-2, 1, len(x))
ffx = lambda x: x+ np.random.normal(0, 10, len(x))
gt = {}
gt['r2'] = []
gt['sl'] = []
gt['_r2'] = []
gt['_sl'] = []
for w in A:
    if w['r2']<90: continue
    i = i +1
    tmp  = w['envelope']
    a = [xx[0] for xx in tmp]
    a = ffx(a)
    b = [xx[1] for xx in tmp]
    b = ff(b)
    gt['r2'].append(w['r2'])
    gt['sl'].append(-w['slope']/100)
    D = lslinear(a,b)
    gt['_r2'].append(D['r2'])
    gt['_sl'].append(D['slope'])
slopes = gt['sl']
r2 = gt['r2']
_slopes = gt['_sl']
_r2 = gt['_r2']



p1 = figure(title="Slope of regressor before/after contamination",tools="save",
            background_fill_color="#E8DDCB")


measured = slopes
hist, edges = np.histogram(measured, density=True, bins=50)


p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649", legend= 'Ground Truth')


measured = _slopes
hist, edges = np.histogram(measured, density=True, bins=50)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#aaabbb", alpha=.8, legend= 'Contaminated')


p1.legend.location = "center_right"
p1.legend.background_fill_color = "darkgrey"
p1.xaxis.axis_label = 'Slope'
p1.yaxis.axis_label = 'Histogram'



p2 = figure(title="Slope Contamination Amplitude", tools="save",background_fill_color="#E8DDCB")



measured = [np.abs(_slopes[i]-slopes[i]) for i in range(len(_slopes))]
hist, edges = np.histogram(measured, density=True, bins=50)


p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#fffaaa", line_color="#033649",alpha = .5)

p2.legend.location = "center_right"
p2.legend.background_fill_color = "darkgrey"
p2.xaxis.axis_label = 'Slope Difference to ground truth'
p2.yaxis.axis_label = 'Histogram'

# -----------------




p3 = figure(title="R2 before/after contamination",tools="save",
            background_fill_color="#E8DDCB")


measured = r2
hist, edges = np.histogram(measured, density=True, bins=50)


p3.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649", legend = "Ground Truth")


p3.legend.location = "center_right"
p3.legend.background_fill_color = "darkgrey"
p3.xaxis.axis_label = 'R2'
p3.yaxis.axis_label = 'Histogram'



p4 = figure(title="Slopes  somewhat connected", tools="save",background_fill_color="#E8DDCB")



measured = _r2
hist, edges = np.histogram(measured, density=True, bins=50)


p3.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#aaabbb", line_color="#033649",alpha=.5, legend = "Contaminated")

p4.legend.location = "center_right"
p4.legend.background_fill_color = "darkgrey"
p4.xaxis.axis_label = 'Slopes'
p4.yaxis.axis_label = 'Histogram'


grid = gridplot([[p1, p2],[p3, None]])
show (grid)

# Step 2: Estimating Ground Truth
Now we want to consider the ground truth slope for X->Y again from contaminated data. We try to methods:
- a : Weighted average of Source specific, taget specific ROI regressor and X-> Y regressor
- b : max R2 among Source specific, taget specific ROI regressor and X-> Y regressor

In [32]:
def fixes(r2,s,n1,n2):
    c1 = R['s-'+n1]
    c2 = R['t-'+n2]
    o1 = None
    o2 = None
    if (r2> c1[0] and r2> c2[0]):
        o1 = s
    elif (c1[0]> r2 and c1[0] > c2[0]):
        o1 = c1[1]
    else:
        o1 = c2[1]
    sumr2 = r2 + c1[0] + c2[0]
    o2 = (c1[1]*c1[0]+c2[0]*c2[1]+s*r2)/sumr2
    return (o1,o2)

ff = lambda x: x+ np.random.normal(-2, 1, len(x))
ffx = lambda x: x+ np.random.normal(0, 10, len(x))
gt = {}
gt['r2'] = []
gt['sl'] = []
gt['_r2'] = []
gt['_sl'] = []
gt['sl_max'] = []
gt['sl_avg'] = []
for w in A:
    if w['r2']<90: continue
    i = i + 1
    tmp  = w['envelope']
    a = [xx[0] for xx in tmp]
    a = ffx(a)
    b = [xx[1] for xx in tmp]
    b = ff(b)
    gt['r2'].append(w['r2'])
    gt['sl'].append(-w['slope']/100)
    D = lslinear(a,b)
    gt['_r2'].append(D['r2'])
    gt['_sl'].append(D['slope'])
    o1, o2 = fixes(D['r2'],D['slope'],w['n1'],w['n2'])
    gt['sl_max'].append(o1)
    gt['sl_avg'].append(o2)

In [33]:
max_method = np.abs(np.array(gt['sl_max']) - np.array(gt['sl']))
avg_method = np.abs(np.array(gt['sl_avg']) - np.array(gt['sl']))
no_method = np.abs(np.array(gt['_sl']) - np.array(gt['sl']))

In [37]:


p1 = figure(title="Slope fixing results",tools="save",
            background_fill_color="#E8DDCB")


measured = max_method
hist, edges = np.histogram(measured, density=True, bins=50)


p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#fffaaa", line_color="#033649", legend= 'Fixed by max method')

measured = no_method
hist, edges = np.histogram(measured, density=True, bins=50)


p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#aaabbb", legend= 'Ground Truth + noise',alpha=.5)

p1.legend.location = "center_right"
p1.legend.background_fill_color = "darkgrey"
p1.xaxis.axis_label = 'Slope difference to ground truth'
p1.yaxis.axis_label = 'Histogram'



p2 = figure(title="Slope fixing results", tools="save",background_fill_color="#E8DDCB")



measured = avg_method
hist, edges = np.histogram(measured, density=True, bins=50)


p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#fffaaa", line_color="#033649",legend= 'Fixed by weighted avg')

measured = no_method
hist, edges = np.histogram(measured, density=True, bins=50)


p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#aaabbb", legend= 'Ground Truth + noise',alpha=.5)

p2.legend.location = "center_right"
p2.legend.background_fill_color = "darkgrey"
p2.xaxis.axis_label = 'Slope Difference to ground truth'
p2.yaxis.axis_label = 'Histogram'

grid = gridplot([[p1, p2]])
show (grid)

In [47]:
print('Median of differences to the ground truth slope (Max Method):')
print(np.median(max_method))
print('Median of differences to the ground truth slope (Weighted AVG Method):')
print(np.median(avg_method))
print('Median of differences to the ground truth slope (No Method used):')
print(np.median(no_method))

Median of differences to the ground truth slope (Max Method):
0.0230334214398
Median of differences to the ground truth slope (Weighted AVG Method):
0.0226925177444
Median of differences to the ground truth slope (No Method used):
0.0443526688706


In [49]:
num.mean(gt['sl'])

-0.10358983975523132

In [50]:
0.022/0.1

0.21999999999999997

In [51]:
np.exp(-.1)

0.90483741803595952

In [52]:
np.exp(-.1+0.022)

0.92496442654353928

In [53]:
np.exp(-.1+0.044)

0.94553913589039629